In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import zipfile
import csv
import pickle

import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import xml.etree.ElementTree as ET

In [2]:
dictionary = pickle.load(open('dictionary.pic', 'rb'))
reverse_dictionary = pickle.load(open('vocabulary.pic', 'rb'))
embeddings = pickle.load(open('embeddings.pic', 'rb'))

In [3]:
test_dataset_path = '../TEST/2017/SemEval2017-Task3-CQA-MD-test-input.xml'

tree = ET.parse(test_dataset_path)
root = tree.getroot()

similarity = []
    
for Question in root:
    QID = int(Question.get('QID'))
    Qtext = Question.find('Qtext').text
    
    embeddings_Qtext = np.mean(np.asarray([embeddings[dictionary[w]] if w in dictionary else np.zeros_like(embeddings[0]) for w in Qtext]), axis=0)
    
    for QApair in Question.iter('QApair'): 
        QAID = int(QApair.get('QAID'))
        QArel = QApair.get('QArel')
        QAquestion = QApair.find('QAquestion').text
        QAanswer = QApair.find('QAanswer').text
        
        embeddings_QAquestion = np.mean(np.asarray([embeddings[dictionary[w]] if w in dictionary else np.zeros_like(embeddings[0]) for w in QAquestion]), axis=0)
        QAconf = np.asscalar(cosine_similarity(embeddings_Qtext.reshape(1, -1), embeddings_QAquestion.reshape(1, -1)))
        
        QApair.set('QArel', 'R' if QAconf > 0.95 else 'I')
        QApair.set('QAconf', str(round(QAconf, 4)))
   
        #    a = embeddings_Qtext
        #    b = embeddings_QAquestion
        #    s = similarity
tree.write('../TEST/2017/SemEval2017-Task3-CQA-MD-test-input-word2vec-webteb.xml', encoding='utf-8')